# Activity Recognition using least PulseWidth

### Get the data ready

In [2]:
#Imports
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import metrics
import os
#Test Commit

### What are our inputs and outputs

In [3]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]


LABELS = [
    "WALKING", 
    "WALKING_UPSTAIRS", 
    "WALKING_DOWNSTAIRS", 
    "SITTING", 
    "STANDING", 
    "LAYING"
] 

### Preparing the datset

In [4]:
DATA_PATH = "data/"
TRAIN  = "train/"
TEST = "test/"
DATASET_PATH = DATA_PATH + "UCI HAR Dataset/" 

def load_X(X_Signal_Paths):
    X_Signals = []
    
    for signal_type_path in X_Signal_Paths:
        file = open(signal_type_path, 'r')
        
        x_temp = []
        for row in file:
            row = row.replace('  ',' ').strip().split(' ')
            x_temp.append(np.array(row, dtype=np.float32))
    
        X_Signals.append(x_temp)        
        file.close()
        
    return np.transpose(np.array(X_Signals), (1,2,0))

X_train_signals_path = [
 DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES]

X_test_signals_path = [
 DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES]

X_train = load_X(X_train_signals_path)
X_test = load_X(X_test_signals_path)

def load_Y(y_path):
    file = open(y_path, 'r')
    
    y_ = np.array([row.replace(' ','').strip().split() for row in file], dtype = np.int32)

    file.close()
    
    return y_ - 1   ## Done for zero indexing


y_train_path = DATASET_PATH + TRAIN + "y_train.txt"

y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_Y(y_train_path)
y_test = load_Y(y_test_path)

### Specifying our parameters

In [5]:
## Characteristics of the input data
training_data_count = len(X_train)  ## 7352 with 50% overlap between each series
test_data_count = len(X_test)  ##2947 testing series
n_steps = len(X_train[0])    ## should be 128 timesteps per series
n_input = len(X_train[0][0])   ## should be 9, 9 kinds of inputs 

## LSTM parameters 
n_hidden = 32  ## Number of features in the hidden layers
n_classes = 6 ## Output classes

## Training HyperParameters 
learning_rate = 0.0025
lambda_loss_amount = 0.0015 ## ToDo
training_iters = training_data_count * 300 ## Loop 300 times over the dataset
batch_size = 1500 
display_iter = 30000

print ("Input characteristics")
print("X Shape: ",X_test.shape)
print("Y Shape: ",y_test.shape)
print("Checking whether the data is normalized")
print("The mean of the data",np.mean(X_test))
print("The std of the data",np.std(X_test))

Input characteristics
X Shape:  (2947, 128, 9)
Y Shape:  (2947, 1)
Checking whether the data is normalized
The mean of the data 0.0991399
The std of the data 0.395671


#### So we have a remaining task of one-hot encoding the output labels. Normalizing of the data is already done

## Building the Network

    First lets define inputs, outputs and weights of the Network

In [6]:
## Placeholders for input and output

x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, n_classes)

## Define Weights and Biases by tf.Variables. 
##Define them using a dictionary, one key for each layer

weights = {
    'hidden': tf.Variable(tf.random_normal([n_input,n_hidden])),
    'out' : tf.Variable(tf.random_normal([n_hidden,n_classes]))
}

biases = {
    'hidden' : tf.Variable(tf.random_normal([n_hidden])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}



In [7]:
x, y, weights, biases

(<tf.Tensor 'Placeholder:0' shape=(?, 128, 9) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(6,) dtype=float32>,
 {'hidden': <tf.Variable 'Variable:0' shape=(9, 32) dtype=float32_ref>,
  'out': <tf.Variable 'Variable_1:0' shape=(32, 6) dtype=float32_ref>},
 {'hidden': <tf.Variable 'Variable_2:0' shape=(32,) dtype=float32_ref>,
  'out': <tf.Variable 'Variable_3:0' shape=(6,) dtype=float32_ref>})

Lets define the function to get the batches 

In [8]:
def getBatchs(X, step, batch_size):
    
    #Transform the input to get more efficientcy
    #TODO
    shape = list(X.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)
    
    for ii in range(batch_size):
        
        index = ((step-1)*batch_size + ii) % len(X)
        batch_s[ii] = X[index]
        
    return batch_s

Also we need one hot encoding to encode the Y

In [9]:
def one_hot(y):
    
    y_ = y.reshape(len(y))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

    Now define the Network

In [10]:
def LSTM_HAR(_X, _weights, _biases):
    """
    Here we use the contrib library in the tensorflow for the RNN implementation
    """
    _X = tf.transpose(_X, [1,0,2])
    
    _X = tf.reshape(_X, [-1,n_input])
    
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    
    _X = tf.split(_X, n_steps, 0)
    
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1,lstm_cell_2], state_is_tuple=True)
    
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)
    
    lstm_last_output = outputs[-1]
    
    return tf.matmul(lstm_last_output,_weights['out']) + _biases['out']
    

In [11]:
pred = LSTM_HAR(x, weights, biases)

l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits=pred)) + l2
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

<tf.Tensor 'Mean_1:0' shape=() dtype=float32>

In [15]:
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()

sess.run(init)

step = 1
while step* batch_size <= training_iters:
    batch_xs = getBatchs(X_train, step, batch_size)
    batch_ys = one_hot(getBatchs(y_train, step, batch_size))
#     print (batch_xs.shape)
#     print(batch_ys.shape)
    
#     assert False
    _, loss, acc = sess.run(
                    [optimizer, cost, accuracy],
                    feed_dict= {
                        x: batch_xs,
                        y: batch_ys
                    }
    )
    train_losses.append(loss)
    train_accuracies.append(acc)
    
    # Evaluate network only at some steps for faster training: 
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):
        
        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc))
        
        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy], 
            feed_dict={
                x: X_test,
                y: one_hot(y_test)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc))

    step += 1

print("Optimization Finished!")

# Accuracy for test data

one_hot_predictions, accuracy, final_loss = sess.run(
    [pred, accuracy, cost],
    feed_dict={
        x: X_test,
        y: one_hot(y_test)
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("FINAL RESULT: " + \
      "Batch Loss = {}".format(final_loss) + \
      ", Accuracy = {}".format(accuracy))

ValueError: Cannot feed value of shape (1500, 6) for Tensor 'Placeholder_1:0', which has shape '(6,)'

In [18]:
tf.placeholder()

TypeError: placeholder() missing 1 required positional argument: 'dtype'